<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=200/>
# ````I. Delta Time Machine````

For reference:
- Featured Summit Talk: https://databricks.com/session_eu20/data-time-travel-by-delta-time-machine-2
- Location of the file in github: https://github.com/vinijaiswal/delta_time_travel

## Topics
Contents of Delta Directory 
1. Data
2. Delta Logs / Transactions
  - LogStore
  - Transaction Protocol
  - Snapshots and Versions 
  - Metadata
  - Checkpoints
4. The History

##Contents of Delta Directory

In [0]:
%fs ls dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2

path,name,size
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/,_delta_log/,0
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00000-418067bd-bf5e-4d28-ac68-ea8cea49b956-c001.snappy.parquet,part-00000-418067bd-bf5e-4d28-ac68-ea8cea49b956-c001.snappy.parquet,542929260
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00000-45b799c4-6305-4c75-a3fd-fa1e840a8b99-c002.snappy.parquet,part-00000-45b799c4-6305-4c75-a3fd-fa1e840a8b99-c002.snappy.parquet,461126371
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00000-626b910a-882d-44ab-badf-756afde9f6e1-c000.snappy.parquet,part-00000-626b910a-882d-44ab-badf-756afde9f6e1-c000.snappy.parquet,540468009
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00001-405b13dc-0f72-419a-ab75-ddf993ba51af-c002.snappy.parquet,part-00001-405b13dc-0f72-419a-ab75-ddf993ba51af-c002.snappy.parquet,541858629
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00001-a794b5c5-8876-4aa9-89c0-92bb15cf5055-c001.snappy.parquet,part-00001-a794b5c5-8876-4aa9-89c0-92bb15cf5055-c001.snappy.parquet,542859315
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/part-00001-c3f739cd-e309-43de-ab84-d005efb6bb7a-c000.snappy.parquet,part-00001-c3f739cd-e309-43de-ab84-d005efb6bb7a-c000.snappy.parquet,541186721


####Note:
By default, the reference implementation stores data files in directories that are named based on the partition values for data in that file (i.e. part1=value1/part2=value2/...).
- The reference implementation uses a Globally Unique Identifier in the name to ensure this property.
- Data files MUST be uniquely named and MUST NOT be overwritten.

##1. DATA

In [0]:
%scala
spark.sql("""
SELECT *
FROM   delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2` 
""").show(5)

+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+---------------+-------+--------------------+------------------+
c_customer_sk| c_customer_id|c_current_cdemo_sk|c_current_hdemo_sk|c_current_addr_sk|c_first_shipto_date_sk|c_first_sales_date_sk|c_salutation|c_first_name|c_last_name|c_preferred_cust_flag|c_birth_day|c_birth_month|c_birth_year|c_birth_country|c_login| c_email_address|c_last_review_date|
+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+---------------+-------+--------------------+------------------+
 35601569|AAAAAAAABKMDPBCA| 103669| 1783| 31530744| 2451663| 2451633| Ms.| Joyce| Woodbury| Y| 7| 1| 1950| NAMIBIA| null|Joyce.Woodbury@r9...| 2452561|
 49826977|AAAAAAAABKMEIPCA| 1764463| 2194| 31181525| 2449658| 2449628| Dr.| Mark| Gaskins| N| 10| 4| 1930| MARTINIQUE| null|Mark.Gaskins@mtDm...| 2452309|
 16534689|AAAAAAAABKMEMPAA| 1482781| 3464| 25531347| 2451051| 2451021| Mr.| Samuel| Sparrow| N| 28| 8| 1975| GUADELOUPE| null|Samuel.Sparrow@3S...| 2452634|
 19946657|AAAAAAAABKMFADBA| 1569629| 5039| 14088096| 2450686| 2450656| Ms.| Winnie| Wilson| Y| 15| 9| 1976| IRELAND| null|Winnie.Wilson@9Jk...| 2452378|
 56974497|AAAAAAAABKMFFGDA| 979209| 1315| 18432574| 2452374| 2452344| Mr.| William| Eubanks| Y| 20| 12| 1988| SINGAPORE| null|William.Eubanks@k...| 2452618|
+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+---------------+-------+--------------------+------------------+
only showing top 5 rows

In [0]:
#Returns information about schema, partitioning, table size, and so on. For example, you can see the current reader and writer versions of a table.
display(spark.sql("Describe Detail delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2`"))

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,4561a292-15fe-4973-b19f-8df92bec9b96,null,null,dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2,2020-05-06T21:41:36.204+0000,2020-06-18T03:28:05.000+0000,List(),6,3170428305,Map(),1,2


In [0]:
## Return the metadata of an existing table (column names, data types, and comments). If the table does not exist, an exception is thrown.
display(spark.sql("describe table delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2`"))

col_name,data_type,comment
c_customer_sk,int,
c_customer_id,string,
c_current_cdemo_sk,int,
c_current_hdemo_sk,int,
c_current_addr_sk,int,
c_first_shipto_date_sk,int,
c_first_sales_date_sk,int,
c_salutation,string,
c_first_name,string,
c_last_name,string,


##2. Delta Logs  / Transactions
- Delta logs are the unit of atomicity for a table, and are named using the next available version number, zero-padded to 20 digits.
- It contains json, crc and checkpoint files

In [0]:
%fs ls dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log

path,name,size
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000010.checkpoint.parquet,00000000000000000010.checkpoint.parquet,120950
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000011.crc,00000000000000000011.crc,96
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000011.json,00000000000000000011.json,83782
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000012.crc,00000000000000000012.crc,96
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000012.json,00000000000000000012.json,43729
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000013.crc,00000000000000000013.crc,96
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000013.json,00000000000000000013.json,58648


###JSON
- Each commit is written out as a JSON file, starting with 000000.json.
- A delta file, n.json, contains an atomic set of actions that should be applied to the previous table state, n-1.json, in order to construct nth snapshot of the table.
- An action changes one aspect of the table's state, for example, adding or removing a file.

In [0]:
%fs head dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000012.json

{"commitInfo":{"timestamp":1588868986897,"userId":"100708","userName":"vini.jaiswal@databricks.com","operation":"DELETE","operationParameters":{"predicate":"[\"exists(t1.`c_customer_id`)\"]"},"notebook":{"notebookId":"6718320"},"clusterId":"0506-044457-pump2","readVersion":11,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numTotalRows":"65000000","numFiles":"23","numRemovedFiles":"31","numCopiedRows":"58500000","numDeletedRows":"6500000","numOutputRows":"42250002","numParts":"0","numOutputBytes":"2245108004","numAddedFiles":"23"}}}
{"remove":{"path":"part-00019-607c1f0e-63e2-4217-8ce7-3dec08c6792b-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00014-2ccbc0c3-05b6-4af0-911d-cc8142de79cc-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00011-c579057b-ad5c-4831-bbd1-de0d2e0a0cb1-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00023-211ec9c4-56cc-447c-8581-bc64a9d5f33e-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00016-247609cc-00fc-4f60-a23a-73c3dce5e1b2-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00003-d3b33690-bf2e-4f6d-9ae5-f65c93cde860-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00004-2740a5a5-454c-4f42-b592-a4038c688e14-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00024-397804ac-7c03-4a12-ac07-b3aabd75be38-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00028-1119fa28-2abf-4fc4-b9b1-80f5c14775df-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00015-71bb39bd-6021-4048-9605-b22071461369-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00013-8ee24c0e-3e0d-4cd3-b47d-f08a72ba695a-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00018-393bb807-b96e-4767-81fb-67b26fdef078-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00030-3fd48453-b87d-427b-b3c5-ae1464ff6b9e-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00027-c391ccb9-aedc-4c74-af24-89333e5704ba-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00026-e0c3f54a-cca5-47e3-a777-3be0a159130e-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00017-9e8d00fc-3829-4265-a988-8187081ed40a-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00005-95b02079-e1e9-4c04-b211-458e0bc8fa53-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00001-6526fa56-eac0-4739-a896-03d65d5ee754-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00012-57f956fa-b8da-4ac3-a040-cfbf052c8697-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00020-6b2f6f85-dc9e-464b-a7a5-6840a465c4bc-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00009-7695d472-54d5-4618-ba71-0f21fa92417e-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00022-e701c7da-ac02-4bd3-8059-e4baadedf9c4-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"remove":{"path":"part-00002-b744628c-9ea3-4bf7-aa2b-ae08c4be13d7-c000.snappy.parquet","deletionTimestamp":1588868986896,"dataChange":true}}
{"add":{"path":"part-00000-1546b333-7a5f-4f23-8501-5ce8c193da20-c000.snappy.parquet","partitionValues":{},"size":67163986,"modificationTime":1588868942000,"dataCha

####CRC files:
- CRC is a file extension for a Cyclical Redundancy Check checksum file used to verify the data integrity of another file. CRC is a popular technique for checking data integrity as it has excellent error detection abilities, uses little resources and is easily used.

In [0]:
%fs head dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000011.crc

{"tableSizeBytes":3453647007,"numFiles":31,"numMetadata":1,"numProtocol":1,"numTransactions":0}

In [0]:
%fs head dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000012.crc

{"tableSizeBytes":3108543540,"numFiles":31,"numMetadata":1,"numProtocol":1,"numTransactions":0}

###3. Checkpoints
- By default, the reference implementation creates a checkpoint every 10 commits.
- Checkpoints are also stored in the _delta_log directory, and can be created for any version of the table.
- The checkpoint file name is based on the version of the table that the checkpoint contains.
- The format of the checkpoint file name can take one of two forms:
- A single checkpoint file for version n of the table will be named n.checkpoint.parquet.
- A multi-part checkpoint for version n can be fragmented into p files. Fragment o of p is named n.checkpoint.o.p.parquet.

In [0]:
 %fs ls dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000010.checkpoint.parquet

path,name,size
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000010.checkpoint.parquet,00000000000000000010.checkpoint.parquet,120950


In [0]:
 %fs ls dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000020.checkpoint.parquet

path,name,size
dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2/_delta_log/00000000000000000020.checkpoint.parquet,00000000000000000020.checkpoint.parquet,94341


###4. The History

Lets look at the historical records by running the describe history command. You can retrieve information on the operations, user, timestamp, and so on for each write to a Delta Lake table.

Using Scala / Java APIs, you can
access the commit history of the table
understand what operations modified the table.
This allows users to maintain a track of all the changes to a table.

In [0]:
##**Note**: The operations are returned in reverse chronological order. By default, table history is retained for 30 days.
display(spark.sql("DESCRIBE HISTORY delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t2`"))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
21,2020-06-18T03:28:05.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0618-012132-spiky36,20,SnapshotIsolation,false,"Map(numRemovedFiles -> 208, numRemovedBytes -> 3194861916, p25FileSize -> 540468009, minFileSize -> 461126371, numAddedFiles -> 6, maxFileSize -> 542929260, p75FileSize -> 542859315, p50FileSize -> 541858629, numAddedBytes -> 3170428305)",null
20,2020-06-18T03:21:57.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0618-012132-spiky36,19,WriteSerializable,false,"Map(numRemovedFiles -> 23, numDeletedRows -> 6500000, numAddedFiles -> 200, numCopiedRows -> 42250002)",null
19,2020-05-07T16:48:32.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,18,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
18,2020-05-07T16:47:05.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,17,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 26, numRemovedFiles -> 9, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 57311759, numParts -> 0, numOutputBytes -> 3041064134, numAddedFiles -> 26)",null
17,2020-05-07T16:45:04.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,16,SnapshotIsolation,false,"Map(numFiles -> 9, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 117798102, minFileSize -> 63127260, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3448893504, numAddedFiles -> 9, maxFileSize -> 530630703, p75FileSize -> 530592677, p50FileSize -> 530560834, numAddedBytes -> 3448893505)",null
16,2020-05-07T16:39:09.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,15,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
15,2020-05-07T16:37:39.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,14,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 26, numRemovedFiles -> 9, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 57314981, numParts -> 0, numOutputBytes -> 3041396932, numAddedFiles -> 26)",null
14,2020-05-07T16:35:35.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,13,SnapshotIsolation,false,"Map(numFiles -> 9, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 136690928, minFileSize -> 62942854, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3449127978, numAddedFiles -> 9, maxFileSize -> 530668371, p75FileSize -> 530603594, p50FileSize -> 530562862, numAddedBytes -> 3449127979)",null
13,2020-05-07T16:31:22.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,12,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
12,2020-05-07T16:29:48.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,11,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 23, numRemovedFiles -> 31, numCopiedRows -> 58500000, num

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=200/>
# ````II. Delta Time Travel````
###Let's ride the Time Machine... let's take you through the adventurous  ride with Delta.

###Why are we time traveling?
1. #####Governance
*Need to protect the people!!!*
2. #####Rollbacks
*To change something that can fix the future!!!*
3. #####Audit / Data Lineage
*Uhhh the audit!!!!*
4. #####Benchmark
*Faster the better!!!*
5. #####Debugging  
*Fix the broken!!!*

###Use Case 1: Governance
######Need to protect the people!!!
For an elaborated session, refer to our past session with notebooks: https://github.com/vinijaiswal/Delta_GDPR_CCPA

In [0]:
#matching records
spark.sql("""
SELECT count(*)
FROM   delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1` 
WHERE  c_customer_id IN (SELECT c_customer_id 
                         FROM   gdpr.customer_delete_keys)""").show()

+--------+
count(1)|
+--------+
 6500000|
+--------+

In [0]:
#removal of records
spark.sql("""
DELETE FROM delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1` AS t1
WHERE  EXISTS (SELECT c_customer_id 
               FROM   gdpr.customer_delete_keys
               WHERE  t1.c_customer_id = c_customer_id)""")

Out[23]: DataFrame[]

In [0]:
## Ensuring the delete operation completion.  Notice that a new version gets added as I performed a Delete operation
display(spark.sql("DESCRIBE HISTORY delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1`"))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
13,2020-10-30T02:37:17.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(7708995),1027-034703-doors186,12,WriteSerializable,false,"Map(numRemovedFiles -> 23, numDeletedRows -> 6500000, numAddedFiles -> 100, numCopiedRows -> 42250002)",null
12,2020-10-30T02:31:36.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(7708995),1027-034703-doors186,11,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647999, numOutputRows -> 65000000)",null
11,2020-10-28T23:33:34.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(7649269),0804-220509-stead130,10,WriteSerializable,false,"Map(numRemovedFiles -> 23, numDeletedRows -> 6500000, numAddedFiles -> 100, numCopiedRows -> 42250002)",null
10,2020-05-07T07:15:57.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6756274),0318-151752-abed99,9,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
9,2020-05-06T23:07:50.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,8,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 27, numRemovedFiles -> 9, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 58500000, numParts -> 0, numOutputBytes -> 3104183076, numAddedFiles -> 27)",null
8,2020-05-06T23:04:48.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,7,SnapshotIsolation,false,"Map(numFiles -> 9, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 126938395, minFileSize -> 62638263, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3448925759, numAddedFiles -> 9, maxFileSize -> 530635226, p75FileSize -> 530603702, p50FileSize -> 530571438, numAddedBytes -> 3448925760)",null
7,2020-05-06T23:00:21.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,6,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
6,2020-05-06T22:49:35.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,5,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 27, numRemovedFiles -> 8, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 58500000, numParts -> 0, numOutputBytes -> 3104110848, numAddedFiles -> 27)",null
5,2020-05-06T22:46:15.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,4,SnapshotIsolation,false,"Map(numFiles -> 8, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 489665779, minFileSize -> 118559808, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3448795229, numAddedFiles -> 8, maxFileSize -> 530687833, p75FileSize -> 530667801, p50FileSize -> 530593058, numAddedBytes -> 3448795230)",null
4,2020-05-06T22:39:27.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,3,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null


###Use case 2: Rollbacks
######To change something that can fix the future!!!

Time travel also makes it easy  to do rollbacks in case of bad writes. For example, if your GDPR pipeline job had a bug that accidentally deleted user information, you can easily fix the pipeline:

- Fixing the accidental deletes to a table for the customer
- Fixing the accidental incorrect updates to a table

In [0]:
df = spark.read.format("delta").option("versionAsOf", "10").load("dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1")
df.write.format("delta").mode("overwrite").save("dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1")

In [0]:
%sql
RESTORE TABLE delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1` TO VERSION AS OF 10


In [0]:
%scala
spark.sql("""
SELECT count(*)
FROM   delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1` 
WHERE  c_customer_id IN (SELECT c_customer_id 
                         FROM   gdpr.customer_delete_keys)""").show()

+--------+
count(1)|
+--------+
 6500000|
+--------+

###Use case 3: Audit / Data Lineage
######Uhhh the audit!!!!

In [0]:
display(spark.sql("DESCRIBE HISTORY delta.`dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t1`"))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
14,2020-10-30T02:50:07.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(7708995),1027-034703-doors186,13,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647999, numOutputRows -> 65000000)",null
13,2020-10-30T02:37:17.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(7708995),1027-034703-doors186,12,WriteSerializable,false,"Map(numRemovedFiles -> 23, numDeletedRows -> 6500000, numAddedFiles -> 100, numCopiedRows -> 42250002)",null
12,2020-10-30T02:31:36.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(7708995),1027-034703-doors186,11,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647999, numOutputRows -> 65000000)",null
11,2020-10-28T23:33:34.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(7649269),0804-220509-stead130,10,WriteSerializable,false,"Map(numRemovedFiles -> 23, numDeletedRows -> 6500000, numAddedFiles -> 100, numCopiedRows -> 42250002)",null
10,2020-05-07T07:15:57.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6756274),0318-151752-abed99,9,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
9,2020-05-06T23:07:50.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,8,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 27, numRemovedFiles -> 9, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 58500000, numParts -> 0, numOutputBytes -> 3104183076, numAddedFiles -> 27)",null
8,2020-05-06T23:04:48.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,7,SnapshotIsolation,false,"Map(numFiles -> 9, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 126938395, minFileSize -> 62638263, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3448925759, numAddedFiles -> 9, maxFileSize -> 530635226, p75FileSize -> 530603702, p50FileSize -> 530571438, numAddedBytes -> 3448925760)",null
7,2020-05-06T23:00:21.000+0000,100708,vini.jaiswal@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(6718320),0506-044457-pump2,6,WriteSerializable,false,"Map(numFiles -> 31, numOutputBytes -> 3453647006, numOutputRows -> 65000000, numParts -> 0)",null
6,2020-05-06T22:49:35.000+0000,100708,vini.jaiswal@databricks.com,DELETE,"Map(predicate -> [""exists(t1.`c_customer_id`)""])",null,List(6718320),0506-044457-pump2,5,WriteSerializable,false,"Map(numTotalRows -> 65000000, numFiles -> 27, numRemovedFiles -> 8, numCopiedRows -> 58500000, numDeletedRows -> 6500000, numOutputRows -> 58500000, numParts -> 0, numOutputBytes -> 3104110848, numAddedFiles -> 27)",null
5,2020-05-06T22:46:15.000+0000,100708,vini.jaiswal@databricks.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""c_customer_id""], batchId -> 0, auto -> false)",null,List(6718320),0506-044457-pump2,4,SnapshotIsolation,false,"Map(numFiles -> 8, numRemovedFiles -> 31, numRemovedBytes -> 3453647007, p25FileSize -> 489665779, minFileSize -> 118559808, numOutputRows -> 65000000, numParts -> 0, numOutputBytes -> 3448795229, numAddedFiles -> 8, maxFileSize -> 530687833, p75FileSize -> 530667801, p50FileSize -> 530593058, numAddedBytes -> 3448795230)",null


###Use case 4: Benchmarking
######Let's increase performance!!!

###Use case 5: Debugging
######Fix the broken!!!

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Vacuum the records for permanent deletion

By default, `vacuum()` retains all the data needed for the last 7 days. For this example, since this table does not have 7 days worth of history, we will retain 0 hours, which means to only keep the latest state of the table.

In [0]:
%sql
VACUUM gdpr.customer_t5 RETAIN 0 HOURS

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.lang.IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
 
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.checkRetentionPeriodSafety(VacuumCommand.scala:78)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.$anonfun$gc$2(VacuumCommand.scala:117)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:412)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:338)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger.recordOperation(UsageLogger.scala:71)
	at com.databricks.spark.util.UsageLogger.recordOperation$(UsageLogger.scala:58)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging.recordOperation(UsageLogger.scala:346)
	at com.databricks.spark.util.UsageLogging.recordOperation$(UsageLogger.scala:325)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordOperation(VacuumCommand.scala:51)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation$(DeltaLogging.scala:94)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordDeltaOperation(VacuumCommand.scala:51)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.$anonfun$gc$1(VacuumCommand.scala:103)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1049)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.gc(VacuumCommand.scala:103)
	at com.databricks.sql.transaction.directory.VacuumTableCommand.$anonfun$run$1(VacuumTableCommand.scala:58)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1067)
	at com.databricks.sql.transaction.directory.VacuumTableCommand.run(VacuumTableCommand.scala:43)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:234)
	at o

In [0]:
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")

Out[37]: DataFrame[key: string, value: string]

In [0]:
%sql
VACUUM gdpr.customer_t5 RETAIN 0 HOURS

path
dbfs:/user/hive/warehouse/gdpr.db/customer_t5


<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=200/>
# ````III: Time Travel options````

Different Ways to do Time Travel
1. Timestamp
2. Parameter "@Timestamp"
3. Version Number
4. Parameter "@Version"

In [0]:
v0 = spark.read.format("delta").option("timestampAsOf", "2020-07-25T22:20:44.000+0000").load(DELTA_SILVER_PATH).count()
vn = spark.read.format("delta").option("timestampAsOf", "2020-07-25T16:28:51.000+0000").load(DELTA_SILVER_PATH).count()
print("Table record counts:\n Past Version [%s] \n Latest Version [%s]" % (v0,vn))

Table record counts:
 Past Version [58500000] 
 Latest Version [58500000]

In [0]:
v0=spark.read.format("delta").load("dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t5@20200725222044000").count()
vn=spark.read.format("delta").load("dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t5@20200725162851000").count() 
print("v0_count: %s \n vn_count: %s" % (v0,vn))

v0_count: 58500000 
 vn_count: 58500000

In [0]:
vl = spark.read.format("delta").option("versionAsOf", 11).load(DELTA_SILVER_PATH).count()
vr = spark.read.format("delta").option("versionAsOf", 8).load(DELTA_SILVER_PATH).count()
print("Table record counts:\n Latest Change [%s] \n Latest Version [%s]" % (vr,vl))

Table record counts:
 Latest Change [58500000] 
 Latest Version [58500000]

In [0]:
spark.read.format("delta").load("dbfs:/Users/vini.jaiswal@databricks.com/demo/customer_t5@v11").count()            # table on version 11

Out[47]: 58500000

###SUMMARY
We learned how time travel is possible with Delta and how it improves developer productivity tremendously.
- Data engineers simplify their pipelines and roll back bad writes.
- Data scientists manage their experiments better.
- Data analysts do easy reporting.
- Organizations can finally standardize on a clean,  centralized, versioned big data repository in their own cloud storage for analytics. 

###References

#####For blogs about using Delta Lake for GDPR and CCPA compliance written by Databricks experts, see:
- For reference of this notebook, visit: https://github.com/vinijaiswal/delta_time_travel
- Watch our other sessions from the Data + AI summit
- To learn more about Delta Lake on Databricks, see <a href="http://docs.databricks.com/delta/index.html">Delta Lake</a>

#####Other References

- <a href="https://docs.databricks.com/security/privacy/gdpr-delta.html">Best Practice Guide</a>
- <a href="https://github.com/databricks/tech-talks">Tech Talk Repo</a>
- <a href="https://www.youtube.com/channel/UC3q8O3Bh2Le8Rj1-Q-_UUbA">Delta YouTube Channel</a>
- <a href="https://www.youtube.com/channel/UC3q8O3Bh2Le8Rj1-Q-_UUbA">Brickstein's Briefcase to hear from our Delta users</a>